In [ ]:
pip install langgraph langchain-openai langchain-community python-docx python-dotenv

In [2]:
from dotenv import load_dotenv
import os
import datetime
from typing import TypedDict, List, Literal, Annotated
from langgraph.graph import StateGraph, END
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import operator

In [3]:

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [4]:
class AgentState(TypedDict):
    conversation_history: Annotated[List[str], operator.add]
    test_report: str
    generated_questions: List[str]
    pending_questions: List[str]
    last_follow_up: datetime.datetime
    symptoms_collected: bool
    report_processed: bool
    next_action: Literal["collect_symptoms", "process_report", "clarify_questions", "follow_up", "await_input", "exit"]
    user_input: str

# Initialize LLMs
llm = ChatOpenAI(temperature=0.2, model="gpt-4o-mini")

In [5]:
def supervisor_node(state: AgentState):
    """ Supervisor decides next action based on user input """
    try:
        # If no input received, wait
        if not state.get("user_input", "").strip():
            print("[Supervisor] Awaiting patient input...")
            return {"next_action": "await_input"}  # NEW state

        # Check conditions to determine next step
        if state.get("pending_questions"):
            return {"next_action": "clarify_questions"}
        elif state.get("test_report") and not state.get("report_processed", False):
            return {"next_action": "process_report"}
        elif state.get("symptoms_collected", False) and state.get("report_processed", False):
            return {"next_action": "follow_up"}

        return {"next_action": "collect_symptoms"}
    
    except Exception as e:
        print(f"[Supervisor] Error: {str(e)}")
        return {"next_action": "await_input"}

def handle_symptoms(state: AgentState):
    """ Collects symptoms from the patient """
    if not state.get("user_input", "").strip():
        return {"next_action": "await_input"}  # NEW state

    messages = [
        SystemMessage(content="Ask patient about symptoms in detail."),
        HumanMessage(content=state["user_input"])
    ]
    
    response = llm.invoke(messages).content
    return {
        "conversation_history": state["conversation_history"] + [f"Assistant: {response}"],
        "symptoms_collected": True,
        "next_action": "supervisor"
    }

def process_test_report(state: AgentState):
    """ Process test report and generate questions """
    try:
        if not state["test_report"]:
            return {"next_action": "await_input"}  # Wait for user to upload report

        loader = Docx2txtLoader(state["test_report"])
        docs = loader.load()
        report_content = docs[0].page_content

        messages = [
            SystemMessage(content="Analyze test report and generate medical questions."),
            HumanMessage(content=report_content)
        ]

        questions = llm.invoke(messages).content.split("\n")
        return {
            "generated_questions": questions,
            "pending_questions": questions,
            "report_processed": True,
            "next_action": "supervisor"
        }
    
    except Exception as e:
        print(f"[Error] Report Processing: {str(e)}")
        return {"next_action": "supervisor"}

def clarify_questions(state: AgentState):
    """ Ask the patient about test report findings """
    if not state["pending_questions"]:
        return {"next_action": "supervisor"}

    current_question = state["pending_questions"].pop(0)
    return {
        "conversation_history": state["conversation_history"] + [f"Assistant: {current_question}"],
        "next_action": "await_input"
    }

def follow_up(state: AgentState):
    """ Follow-up on patient's progress """
    messages = [
        SystemMessage(content="Generate follow-up questions based on history."),
        HumanMessage(content="\n".join(state["conversation_history"]))
    ]

    follow_up_question = llm.invoke(messages).content
    return {
        "conversation_history": state["conversation_history"] + [f"Assistant: {follow_up_question}"],
        "last_follow_up": datetime.datetime.now(),
        "next_action": "await_input"
    }


In [6]:

# Build the new workflow graph
workflow = StateGraph(AgentState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("collect_symptoms", handle_symptoms)
workflow.add_node("process_report", process_test_report)
workflow.add_node("clarify_questions", clarify_questions)
workflow.add_node("follow_up", follow_up)

# NEW: `await_input` state to wait for patient input before proceeding
workflow.add_node("await_input", lambda state: {"next_action": "supervisor"})

# Supervisor decides next step
workflow.add_conditional_edges(
    "supervisor",
    lambda state: state["next_action"],
    {
        "collect_symptoms": "collect_symptoms",
        "process_report": "process_report",
        "clarify_questions": "clarify_questions",
        "follow_up": "follow_up",
        "await_input": "await_input",
        "exit": END
    }
)

# Each step returns to supervisor after completion
for node in ["collect_symptoms", "process_report", "clarify_questions", "follow_up"]:
    workflow.add_edge(node, "supervisor")

workflow.set_entry_point("await_input")  # Start by waiting for user input
agent = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

In [ ]:
# Chat interface
def chat_interface():
    """ Main chatbot loop """
    state = {
        "conversation_history": ["Assistant: Hello! I'm your health assistant. Let's start with your symptoms."],
        "test_report": "",
        "generated_questions": [],
        "pending_questions": [],
        "last_follow_up": None,
        "symptoms_collected": False,
        "report_processed": False,
        "next_action": "await_input",
        "user_input": ""
    }

    print(state["conversation_history"][0])

    while True:
        try:
            if state["next_action"] == "await_input":
                user_input = input("\nPatient: ")
                state["user_input"] = user_input.strip()
            elif state["next_action"] == "process_report":
                report_path = input("\n[System] Upload test report path: ")
                state["test_report"] = report_path.strip()
                state["user_input"] = "[REPORT_UPLOADED]"

            result = agent.invoke(state)
            state.update(result)

            if state["conversation_history"]:
                print(f"\nAssistant: {state['conversation_history'][-1]}")

            if state.get("next_action") == "exit":
                print("\n[Doctor's Summary]:", generate_summary(state))
                break

        except Exception as e:
            print(f"Error: {str(e)}")
            state["next_action"] = "await_input"

if __name__ == "__main__":
    chat_interface()
